# SPRING2022 – Online Summerschool Python für Ingenieurinnen, 2022

## Aufgabensammlung (Teil 2)

Dieses Notebook enthält Anregungen für Übungsaufgaben.
Hinweise zur Lösung können dem Kursmaterial (Folien), dem Notebook [00_demo](00_demo.ipynb) und den aufgabenspezifischen Tipps entnommen werden.

**Empfehlung**: Musterlösung nicht kopieren, sondern:
1. Mindestens 2min überlegen und einfach mal was ausprobieren,
2. Tipps durchlesen dann nochmal überlegen, probieren, recherchieren
3. ggf. Musterlösung anschauen, verstehen und dann wieder zuklappen. Dann aus dem Gedächtnis in die Code-Zelle übertragen (bei Bedarf nachschauen). **Code selber schreiben** ist didaktisch viel wertvoller als kopieren!
4. Alternative Lösungswege oder Modifikationen durchdenken

In [ ]:
# Diese Zelle importiert und initialisiert alles was für die Aufgaben in diesem Notebook wichtig ist.
import numpy as np
import sympy as sp
import pandas as pd
from sympy.interactive import printing
printing.init_printing()
import matplotlib.pyplot as plt
from IPython.display import display
%matplotlib inline

---
---

### Aufgabe 5.1 (Sympy)

* a) Berechnen Sie die 1. und 2. Ableitung der (parameterabhängigen) Funktion $f(x) = \dfrac{\sin(x)}{x^a + 1}$ symbolisch,
* b) bringen sie alle Ausdrücke jeweils auf einen Nenner und
* c) stellen Sie für $a = 2$ die Funktionen $f, f'$ und $f'' $ Ergebnis im Intervall $x\in [-3, 3]$ grafisch dar.



---

<details>

<summary>→ Tipps ▼</summary>

- `sp.` → Zugriff auf SymPy-Modul (siehe Import oben)
- `sp.symbols("X, Y, Z")`: Symbole anlegen, `sp.sin(...)`: Symbolische Sinus-Funktion,
- `f.diff?`: Ableitungen des Ausdrucks `f` berechnen
- `sp.plot?`: Plot-Funktion von sympy (benutzt intern Matplotlib)
</details>

---

<details>

<summary>→ Musterlösung ▼</summary>
    
```python
a, x = sp.symbols("a, x")

f = sp.sin(x)/(1+ x**a)

# Ableitungen berechnen
f1 = f.diff(x)
f2 = f.diff(x, 2)

# Ausgabe
print("Ableitungen (noch nicht vereinfacht):")
display(sp.Eq(sp.Symbol("f'"), f1))
display(sp.Eq(sp.Symbol("f''"), f2))

# Vereinfachen
f1 = f1.simplify() 
f2 = f2.simplify()

# Ausgabe
print("Ableitungen (vereinfacht):")
display(sp.Eq(sp.Symbol("f'"), f1))
display(sp.Eq(sp.Symbol("f''"), f2))

# a = 2 einsetzen
g0 = f.subs(a, 2)
g1 = f1.subs(a, 2)
g2 = f2.subs(a, 2)

# Plot-Funktion von sympy verwenden (benutzt intern Matplotlib)
sp.plot(g0, g1, g2, (x, -3, 3))
```
</details>

In [ ]:
# Hier bitte Lösung einfügen






<br><br><br><br><br><br>

Denkanstöße und Erweiterungen:
- Wie ließe sich $f$ für gegebene Werte von $a$ und $x$ numerisch auswerten?
- Wie ließen sich die algorithmisch die Nullstelle(n) von  $f, f'$ und $f'' $ berechnen?

---
---

### Aufgabe 6.1 (Scipy, Simulation)

Gegeben ist ein mechanisches System bestehend aus zwei visko-elastisch gekoppelten Körpern:

<img src="img/wagen_feder_wagen.svg.png" alt="Dynamisches System (2 gekoppelte Wagen)" title="Dynamisches System (2 gekoppelte Wagen)"> 

Die Bewegungsgleichungen für dieses System lauten:

$$
\begin{align}
%\label{eq_}
m_1 \ddot q_1 + d_1 (\dot q_1 - \dot q_2) + c_1 (q_1 - q_2) &= F\\ 
m_2 \ddot q_2 - d_1 (\dot q_1 - \dot q_2) - c_1 (q_1 - q_2) &= 0.
\end{align}
$$


Simulieren Sie (durch Anpassung des Notebooks [Simulation_dynamischer_Systeme.ipynb](./zusatz-notebooks/Simulation_dynamischer_Systeme.ipynb)) die Bewegung des Systems und visualisieren Sie die Zeitverläufe von $q_1(t), q_2(t)$ für $t \in [0s, 7s]$. Es gelten folgende Annahmen:

- Das System befindet sich am Anfang in Ruhe bei $q_1(t) = q_2(t) = 0$
- Der Verlauf der äußeren Kraft $F$ ist
$F = \cases{\sin(\pi\cdot t)\enspace \text{für} ~t \in [0s, 2s],\\ 0 \hspace{16mm} \text{sonst}}$
- Die Paramter haben folgende (normierte) Werte: $m_1 = 1$, $m_2 = 0.5$, $c_1 = 10$, $d_1 = 0.3$

---

<details>

<summary>→ Tipps ▼</summary>

- Lösen Sie (ggf. auf Papier) die Bewegungsgleichungen nach den Beschleunigungen auf
- Nutzen Sie $\mathbf x = (x_1, x_2, x_3, x_4)^T = (q_1, q_2, \dot q_1, \dot q_2)^T$ als Definition des Zustandsvektors
- Schreiben Sie eine Funktion `rhs(t, xx)`, welche zu einem gegebenen Zeitpunkt `t` für einen gegebenen Zustandsvektor $\mathbf x=$`xx` die zugehörige Zeitableitung des Zustandsvektors $\ddot {\mathbf{x}}=(\dot x_1, \dot x_2, \dot x_3, \dot x_4)^T = (\dot q_1, \dot q_2, \ddot q_1, \ddot q_2)^T$=`xxdot` ausrechnet und zurückgibt (`return`).
- Nutzen Sie `solve_ivp` wie im Beispiel Notebook.
</details>

---

<details>

<summary>→ Musterlösung ▼</summary>
    
```python
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt

# Parameter festlegen:
m1 = 1
m2 = 0.5
c1 = 10
d1 = 0.3


# Definition der rechten Seite (f) der Zustands-DGL

def rhs(t, xx):
    """
    
    :param t:   Auswertezeitpunkt (wichtig für F(t))
    :param xx:  Zustandsvektor    
    :returns:   time derivative of the state vector (as 1D-array)
    """
    q1, q2, q1dot, q2dot = xx
    
    # Boolsche Werte können in numerischen Rechnungen
    # als 1 (True) oder 0 (False) verwendet werden
    F_condition = t<2
    F = np.sin(np.pi*t)*F_condition
    
    # Geschwindigkeiten (definitorische Gleichungen)
    x1dot = q1dot
    x2dot = q2dot
    
    # Beschleunigungen (umgestellte Bewegungsgleichungen)
    x3dot = (F - d1*(q1dot - q2dot) - c1*(q1 - q2))/m1
    x4dot = (d1*(q1dot - q2dot) + c1*(q1 - q2))/m2
    
    # Zeitableitung des Zustandsvektors
    xxdot = np.array([x1dot, x2dot, x3dot, x4dot])
    return xxdot

# Anfangszustand
xx0 = [0, 0, 0, 0]

# Auswertezeitpunkte (20s)
tt = np.linspace(0, 7, 1000)
res = solve_ivp(rhs, (tt[0], tt[-1]), xx0, t_eval=tt, max_step=0.1)

# Erinnerung: res.y ist ein 2D-Array des Simulationsergebnisses
# (Zeilen: Zustandskomponenten; Spalten: Zeitpunkte)
plt.plot(tt, res.y[0, :], label="$q_1$(t)")
plt.plot(tt, res.y[1, :], label="$q_2$(t)")
plt.legend()
```
</details>


In [ ]:
# Hier bitte Lösung einfügen






<br><br><br><br><br><br>

Denkanstöße und Erweiterungen:
- Was passiert bei einer zeitlich konstanten Kraft $>0$?
- Wie können Sie (in einer separaten Abbildung) die beiden Geschwindigkeiten und die Relativgeschwindigkeit der beiden Wagen visualisieren?
- Wie können Sie die an das System übertragene Leistung und die im System gespeicherte Energie (potentiell und kinetisch) berechnen und visualisieren?

---
---

### Aufgabe 6.2 (Scipy, Optimierung)


Der folgende Code repräsentiert mit der Funktion $\mathbf f$ ≙`f` die Temperaturverteilung auf der Wärmebildaufnahme eines Werkstücks. Die Funktion bildet 2D-Vektoren
$\left(\begin{smallmatrix} x\\ y \end{smallmatrix} \right) \in \mathbb{R}^2$ auf einen 1D Temperaturwert $z \in \mathbb{R}$ ab und ist im Bereich $-1 \leq x, y \leq 1$ grafisch dargestellt.


In [ ]:
def f(xx):
    """
    Funktion der Temperaturverteilung (ersetzt Messdaten)
    """
    
    hull = -30*np.exp(-(xx[0]**2+ xx[1]**2)*2)
    return  hull*(5*(np.sin(4*xx[0] + 6.1*xx[1]**2) + np.sin(3*xx[1]))) + 300

# 2D Arrays mit X,Y-Werten Vorbereiten (muss man jetzt nicht im Detail verstehen)
N = 2000
xxN = np.linspace(-1, 1, N)
yyN = np.linspace(-1, 1, N)
XX, YY = np.meshgrid(xxN, yyN)

# Funktion auswerten
ZZ = f((XX, YY))

# Visualisierung (muss man jetzt nicht im Detail verstehen)
fig = plt.figure(figsize=(15, 5))
ax1 = fig.add_subplot(121)
plt.sca(ax1)
plt.contourf(XX, YY, ZZ, 100, cmap="plasma")
plt.colorbar()
plt.xlabel("x"); plt.ylabel("y");

# Demo: einen einzelnen Punkt plotten
plt.plot([-0.75], [0.5], marker="x", color="y")

ax2 = fig.add_subplot(122, projection='3d', azim=45)
ax2.plot_wireframe(XX, YY, ZZ, rstride=70, cstride=70)
ax2.set_xlabel("x"); ax2.set_ylabel("y"); ax2.set_zlabel("z")


Finden Sie mit Hilfe der Funktion [`scipy.optimize.minimize`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html) die $(x, y)$-Koordinaten des Temperaturminimums auf dem Werkstück und zeichnen Sie diesen Punkt im "Wärmebild" ein. Lesen Sie dazu eine geeignete Startschätzung aus der Grafik ab.
 
---

<details>

<summary>→ Tipps ▼</summary>

- Entnehmen Sie der [Dokumentation von minimize](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html) wie Sie die `minimize`-Funktion importieren und welche Argumente diese Funktion unbedingt erwartet.
- Lassen Sie sich das Ergebnis von `res = minimize(...)` anzeigen und überlegen Sie, welches Attribut für die Aufgabe relevant ist. Tipp: es ist nicht `res.message`.
- Nutzen Sie den Code für den Demo-Punkt für die Visualisierung
</details>

---

<details>

<summary>→ Musterlösung ▼</summary>
    
```python
# Optimierungsproblem lösen

from scipy.optimize import minimize
res = minimize(f, x0=[0.5, 0.5])

# Visualisierung Temperaturverteilung (kopiert)
plt.contourf(XX, YY, ZZ, 100, cmap="plasma")
plt.colorbar()
plt.xlabel("x"); plt.ylabel("y");

# Minimum eintragen
plt.plot([res.x[0]], [res.x[1]], marker="x", color="y")
```
</details>

In [ ]:
# Hier bitte Lösung einfügen






<br><br><br><br><br><br>

Denkanstöße und Erweiterungen:
- Inwiefern hängt das Optimierungsergebnis von der Startschätzung ab? (Ausprobieren!)
- Wie lässt sich für die beiden heißen Bereiche jeweils der Punkt der maximalen Temperatur bestimmen?

---
---

### Aufgabe 6.3 (numpy, matplotlib)


Das Array `ZZ` (`shape`: $2000 \times 2000$) aus der vorherigen Aufgabe enthält die Temperaturvereilung des Werkstücks. Berechnen Sie daraus mittels boolscher-Array-Operationen näherunsweise den Anteil der Fläche dessen Temperatur kleiner als $280°\mathrm{C}$ ist und stellen Sie die Bereiche grafisch dar.

 
---

<details>

<summary>→ Tipps ▼</summary>

- `np.sum()` Berechnet die Summe aller Werte in einem Array
- Erinnerung: Vergleichsoperationen wie `<` `>`, `==` mit Numpy-Arrays werden *elementweise* durchgeführt: Das Ergebnis ist wieder ein Array gleicher Form (shape).
- Boolsche Werte (`False` bzw. `True`) werden in numerischen Rechnungen als `0` oder `1` interpretiert.
</details>

---

<details>

<summary>→ Musterlösung ▼</summary>
    
```python
# Variante 1: mit Zwischenergebnissen (leichter verständlich) 

mask = ZZ < 280

# mask ist ein (2000x2000)-Array mit True und False-Werten (je nach Temperaturwert in Z)
# Trick: True als 1 und False als 0 interpretieren
# -> Array-Zellen zählen, für die Bedingung (< 280) erfüllt war
cool_cell_number = np.sum(mask)

# Anteil an allen Zellen bestimmen:
fraction_of_cool_cells = cool_cell_number/mask.size


# Variante 2 als kurzer Einzeiler:
print("Fraction of cool cells:", np.sum((ZZ < 280)/ZZ.size))


# Visualisierung
plt.contourf(XX, YY, ZZ < 280, 100, cmap="plasma")
plt.colorbar()
```
</details>

In [ ]:
# Hier bitte Lösung einfügen






<br><br><br><br><br><br>

Denkanstöße und Erweiterungen:
- Wie ließe sich der Durchschnitt aller Temperaturen berechnen, die größer als $280°\mathrm{C}$ sind?